In [66]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split

from scipy.linalg import svd
from scipy.sparse import csr_matrix

from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
articles_df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f0e8eb4ddd03415fdd4db4a89a2b0d3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/shared_articles.zip')
interactions_df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/186647c8bd3fdb43b78fbc84ace97aed/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/users_interactions.zip')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [7]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [8]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [9]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [10]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [11]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [12]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [13]:
split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


**Задание 6.1**

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

In [14]:
ratings = pd.pivot_table(
    interactions_train_df,
    values="eventStrength",
    index="personId",
    columns="contentId",
).fillna(0)
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

In [25]:
# Преобразуем датасет в массив Numpy для ускорения роботы
ratings_np = ratings.to_numpy()

**Задание 6.2**

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [30]:
ratings_np.mean().round(3)

0.017

**Задание 6.3**

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.



In [34]:
similarity_users = np.zeros((len(ratings_np), len(ratings_np)))
for i in (range(len(ratings_np)-1)):
    for j in range(i+1, len(ratings_np)):
     
        mask_uv = (ratings_np[i] != 0) & (ratings_np[j] != 0)
        ratings_v = ratings_np[i, mask_uv]
        ratings_u = ratings_np[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40].round(2)

-0.33

**Задание 6.4**

Постройте рекомендательную систему по алгоритму (для каждого пользователя):
1. Найти пользователей с похожестью больше 0.
2. Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел). описанному выше. 

Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [44]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)
 
interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)
 
interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_np[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based

prediction_user_based[35][0]

'-5148591903395022444'

In [45]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

**Задание 6.5**

Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [46]:
calc_precision('prediction_user_based').round(3)

0.005

**Задание 6.6**

Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [50]:
U, sigma, V = svd(ratings)
U.max().round(2)

0.71

**Задание 6.7**

Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [60]:
s = np.diag(sigma[:100])
s.sum().round(2)

2096.43

In [62]:
new_ratings = pd.DataFrame(U[:, :100].dot(s).dot(V[:100]), index=ratings.index, columns=ratings.columns)

new_ratings

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9217155070834564627,921770761777842242,9220445660318725468,9222265156747237864,943818026930898372,957332268361319692,966067567430037498,972258375127367383,980458131533897249,98528655405030624
personId,,,,,,,,,,,,,,,,,,,,,
-1007001694607905623,-0.003787,0.000872,-0.001285,0.002931,0.003889,0.013863,-0.001591,0.032587,-0.001272,0.000256,...,-0.003759,-0.000227,-0.008608,-0.000033,0.010733,0.009452,0.003438,-0.000179,0.009597,-0.002503
-1032019229384696495,0.992435,0.015898,0.030610,0.001633,0.012849,0.033984,1.013460,0.002882,0.010873,-0.004411,...,2.974838,0.007639,-0.020085,-0.004258,2.353326,-0.031199,0.033007,0.002056,-0.003534,0.009510
-108842214936804958,-0.015866,-0.029223,-0.032987,-0.012270,0.045642,0.015761,0.177709,0.014010,0.021468,-0.029695,...,0.134796,0.029271,-0.050596,-0.000277,-0.178115,-0.080439,1.891358,0.016009,0.003305,-0.017166
-1130272294246983140,0.166369,0.106842,0.223735,0.055399,-0.009947,-0.088267,0.004609,-0.070600,-0.039416,0.062143,...,0.042020,-0.058595,0.214094,-0.005800,1.156069,-0.176958,-0.020508,0.041146,-0.055612,0.017217
-1160159014793528221,0.011422,-0.002571,0.004961,-0.005116,-0.004820,-0.051882,-0.016217,0.000016,0.019629,-0.003028,...,-0.004893,0.008450,-0.024916,0.002783,-0.007486,0.035766,0.003191,0.000242,-0.034082,0.003778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953707509720613429,-0.008445,0.008741,-0.034499,0.012429,-0.004019,-0.068137,-0.026000,-0.008958,-0.006899,0.010585,...,-0.008839,-0.023717,-0.067747,0.000863,0.157679,-0.019117,-0.008760,-0.001366,0.075268,0.004712
983095443598229476,-0.033461,-0.017441,0.003829,-0.024063,-0.079712,-0.018086,-0.057891,-0.020028,-0.005829,0.058693,...,0.020201,-0.012622,-0.050680,0.035532,-0.107726,0.088145,0.043583,0.010233,-0.078222,0.011099
989049974880576288,-0.012545,-0.017248,0.010352,-0.005367,-0.035086,0.094543,-0.045640,0.004211,-0.007016,0.050335,...,-0.000913,-0.053048,0.207418,0.032478,-0.205465,0.077313,0.040472,0.013610,0.181102,0.004919


**Задание 6.8**

Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [65]:
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd").round(3)

0.012

In [67]:
ratings_matrix = csr_matrix(ratings)

**Задание 6.9**

Воспользовавшись функцией random_train_test_split() из библиотеки lightfm, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра random_state возьмите число 13.

Обучите модель LightFM со 100 компонентами, параметром random_state = 13, темпом обучения 0.05 и функцией потерь 'warp'. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (precision@k) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  
from scipy.sparse import csr_matrix

ratings_matrix = csr_matrix(ratings) # передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)
prec_score = precision_at_k(model, test).mean()


print(round(prec_score, 2))

In [68]:
(1+1/4+1/3)/3

0.5277777777777778